In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.integrate import solve_ivp
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

Dataset Generation

In [31]:
G=6.67430e-11

def newton_equations(t,y,m1,m2):         ##x_1=(x1,y1)  x_2=(x2,y2)  v_1=(vx1,vy1)  v_2=(vx2,vy2)
  x1,y1,x2,y2,vx1,vx2,vy1,vy2=y;
  dx=abs(x2-x1);
  dy=abs(y2-y1);

  r=np.sqrt(dx**2+dy**2);

  x_1=np.array([x1,y1]);
  x_2=np.array([x2,y2]);
  v_1=np.array([vx1,vy1]);                                          #v_1=dx_1/dt  v_2=dx_2/dt
  v_2=np.array([vx2,vy2]);

# Calculating accelerations of both planets
  ax1= -G*m2*dx/r**3;                                              #a_1= dv_1/dt   a_2=dv_2/dt
  ay1= -G*m2*dy/r**3;
  ax2=  G*m1*dx/r**3;
  ay2=  G*m1*dy/r**3;

  return [vx1,vy1,vx2,vy2,ax1,ay1,ax2,ay2];




In [32]:
def trajectory(x_1, x_2, v_1, v_2, t_max,n, m1,m2):

  y0=np.concatenate([x_1,x_2, v_1,v_2]);
  t_span=[0,t_max];
  t_eval=np.linspace(0,t_max,n);

  soln= solve_ivp(newton_equations,t_span,y0,t_eval=t_eval, args=(m1,m2), rtol=1e-10, atol=1e-10);

  traj = { 'time': soln.t, 'positions': soln.y[:4].T, 'velocities': soln.y[4:].T };

  return traj;




In [33]:
def dataset(n):            # number of different cases we want
  dataset=[];
  t_max=1000;

  for i in range(n):                         #iterate to generate different set of initial conditions
    m1=np.random.uniform(1e23,1e30);         #generate random values of masses of planets 1 and 2
    m2=np.random.uniform(1e23,1e30);
    x_1= np.random.uniform(-1e12,1e12,2);       #position of planet 1
    x_2= x_1 + np.random.uniform(-1e12,1e12,2);    #position of planet 2. Added to ensure enough distance between two planets

   # We have to define the velocities such that the two body system follows laws of physics. So we cannot just generate random values of velocities.
    r=x_2-x_1;
    r_magnitude=np.sqrt(r[0]**2+r[1]**2);
    v_magnitude= np.sqrt(G*(m1+m2)/r_magnitude);
    v_1 = np.random.uniform(-1, 1, 2) * v_magnitude;
    v_2 = -v_1 * m1/m2;

    traj= trajectory(x_1,x_2,v_1,v_2, t_max, n,m1,m2);

    for t, pos in zip(traj['time'], traj['positions']):
     X_datapoint= [m1, m2, *v_1, *v_2,*x_1, *x_2, t];
     y_datapoint=pos;
     dataset.append((X_datapoint, y_datapoint));

  return dataset;



Model Definition

In [34]:
full_dataset = dataset(n=100)
X=np.array([data[0] for data in full_dataset]);
y=np.array([data[1] for data in full_dataset]);
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=42);

In [35]:
#since the planets are in a two body system their position will be a periodic function and will repeat after each revolution. Thus, we use fourier features

def fourier_features(X, n_terms):        # adding fourier features to the existing features
  t=X[:,-1];
  t_max=np.max(t);
  new_feature=[];
  #Planet's position can be approximated as x(t)≈ a1sin(2πt/T) + b1cos(2πt/T) + a2sin(4πt/T)+ ....

  for n in range(1,n_terms+1):
    new_feature.append(np.sin(n*np.pi*t/t_max));
    new_feature.append(np.cos(n*np.pi*t/t_max));
  return np.column_stack([X, *new_feature]);


In [36]:
X_train_fourier=fourier_features(X_train,n_terms=5);
X_test_fourier=fourier_features(X_test,n_terms=5);

In [37]:
model= LinearRegression()                 #Linear regression used to learn the coefficients an, bn to fit the expression for x(t)
model.fit(X_train_fourier, y_train)

LinearRegression()

In [38]:
train_pred=model.predict(X_train_fourier);
test_pred=model.predict(X_test_fourier);

In [39]:
def predict_positions(m1, m2, r1_0, r2_0, v1_0, v2_0, t):

    x = np.array([m1, m2, *r1_0, *r2_0, *v1_0, *v2_0, t])
    x_fourier = fourier_features(x.reshape(1, -1))
    return model.predict(x_fourier)[0]

# Physics equations
    r = sp.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    ax1_true = G * m2 * (x2 - x1) / r**3                #true acceleration
    ay1_true = G * m2 * (y2 - y1) / r**3

In [40]:
import sympy as sp
import numpy as np

def compute_ode_residual(model, X_sample, n_terms=3):

    #Computing ODE residual for all 4 position coordinates (x1, y1, x2, y2).

    m1, m2 = X_sample[0], X_sample[1]
    t_sample = X_sample[-1]
    t = sp.symbols('t')

    # Extract all model coefficients
    coeffs = model.coef_.flatten()
    intercepts = model.intercept_
    n_coeffs_per_coord = 1 + 2 * n_terms  # 1 constant + 2*n_terms sin/cos

    # Split coefficients for each coordinate
    x1_coeffs = np.concatenate([[intercepts[0]], coeffs[:n_coeffs_per_coord-1]])
    y1_coeffs = np.concatenate([[intercepts[1]], coeffs[n_coeffs_per_coord-1 : 2*(n_coeffs_per_coord-1)]])
    x2_coeffs = np.concatenate([[intercepts[2]], coeffs[2*(n_coeffs_per_coord-1) : 3*(n_coeffs_per_coord-1)]])
    y2_coeffs = np.concatenate([[intercepts[3]], coeffs[3*(n_coeffs_per_coord-1):]])

    # Build symbolic expressions for each coordinate
    def fourier_expr(coeffs):
        expr = coeffs[0]  # Constant term
        for n in range(1, n_terms+1):
            expr += coeffs[2*n-1] * sp.sin(2*sp.pi*n*t) + coeffs[2*n] * sp.cos(2*sp.pi*n*t)
        return expr

    x1 = fourier_expr(x1_coeffs)
    y1 = fourier_expr(y1_coeffs)
    x2 = fourier_expr(x2_coeffs)
    y2 = fourier_expr(y2_coeffs)

    # Compute second derivatives (predicted accelerations)
    d2x1dt2 = sp.diff(x1, t, 2)
    d2y1dt2 = sp.diff(y1, t, 2)
    d2x2dt2 = sp.diff(x2, t, 2)
    d2y2dt2 = sp.diff(y2, t, 2)

    # Physics-correct accelerations (Newton's gravitation)
    dx = x2 - x1
    dy = y2 - y1
    r = sp.sqrt(dx**2 + dy**2)

    ax1_true = G * m2 * dx / r**3
    ay1_true = G * m2 * dy / r**3
    ax2_true = -G * m1 * dx / r**3  # Equal and opposite force
    ay2_true = -G * m1 * dy / r**3

    # Residuals (difference between predicted and physics accelerations)
    res_x1 = (d2x1dt2 - ax1_true)**2
    res_y1 = (d2y1dt2 - ay1_true)**2
    res_x2 = (d2x2dt2 - ax2_true)**2
    res_y2 = (d2y2dt2 - ay2_true)**2

    # Substitute numerical values and evaluate
    residual_values = []
    for residual in [res_x1, res_y1, res_x2, res_y2]:
        residual_num = residual.subs(t, t_sample).evalf()
        residual_values.append(float(abs(residual_num)))

    return np.mean(residual_values)  # Mean ODE violation

In [41]:
def predict_positions(m1, m2, r1_0, r2_0, v1_0, v2_0, t):
    x = np.array([m1, m2, *r1_0, *r2_0, *v1_0, *v2_0, t])
    x_fourier = fourier_features(x.reshape(1, -1), n_terms=5)
    return model.predict(x_fourier)[0]

# Example prediction
pred = predict_positions(m1=1e28, m2=1e28, r1_0=[-1e10, 0], r2_0=[1e10, 0],  v1_0=[0, -1e4], v2_0=[0, 1e4],t=50)
print(f"Predicted positions at t=50: {pred}")

Predicted positions at t=50: [ 8.35148026e+10 -1.45735134e+11  1.70086191e+11 -2.09071208e+11]


Evaluation Criteria and Final Model Performance

In [42]:
print(compute_ode_residual(model, X_test[0]))

3.0421817716098763e-06


In [43]:
print(f"Train MAE: {mean_absolute_error(y_train, train_pred):.2e}")
print(f"Test MAE: {mean_absolute_error(y_test, test_pred):.2e}")

Train MAE: 6.28e+11
Test MAE: 6.26e+11
